In [1]:
import os
import io
import datetime
import logging
import sys

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 

import civis
import civis.io
from civis.futures import CivisFuture

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, RandomForestRegressor
from civis.ml import ModelPipeline

import uuid
import json
from pprint import pprint
import tempfile
import concurrent.futures
from concurrent.futures import wait
from collections import namedtuple

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Default feature lists for Rainbow Modeling Frame (each number corresponds to number of features)
feature_table = civis.io.read_civis_sql(sql='select * from bernie_nmarchio2.feature_list order by sort_order asc', use_pandas = True, database='Bernie 2020')
feature_list_large = list(feature_table[(feature_table['frame_large'] == 1)]['feature_name']) + ['state_code']
feature_list_medium = list(feature_table[(feature_table['frame_medium'] == 1)]['feature_name']) + ['state_code']
feature_list_small = list(feature_table[(feature_table['frame_small'] == 1)]['feature_name']) + ['state_code']

table_columns = civis.io.read_civis_sql(
    sql=f'''select ordinal_position as position, column_name, data_type 
    from information_schema.columns 
    where table_name = 'rainbow_modeling_frame' and table_schema = 'bernie_data_commons' and column_name != 'person_id'
    order by ordinal_position;''', use_pandas = True, database='Bernie 2020')

#exclusion_list_466 = [e for e in list(table_columns['column_name']) if e not in feature_list_466] 

In [3]:
# Set Parameters
DATABASE = 'Bernie 2020'

# Primary key in both the DV table and the Modeling Frame
PRIMARY_KEY = 'person_id'

# Table containing recoded Dependent Variables keyed to the PRIMARY_KEY
DV_TABLE = 'bernie_nmarchio2.spoke_dvs'
# List of binarized dependent variables (accepts 1, 0, and null values) in DV_TABLE
DV_LIST = ['spoke_support_1box', 'spoke_persuasion_1plus', 'spoke_persuasion_1minus']


# Table containing covariates and keyed to PRIMARY_KEY
MODELING_FRAME = 'bernie_data_commons.rainbow_modeling_frame'
# Columns in the Modeling Frame to exclude from feature list (i.e., strings or incomplete coverage)
EXCLUSION_COLUMNS = ['state_code']

# Schema to contain prediction tables
SCHEMA = 'bernie_nmarchio2'
# String that will be concatenated in front of the output table's name
PREFIX = 'scored'


In [4]:
datestamp = '{:%Y%m%d}'.format(datetime.date.today())

In [5]:
# Counts of positive and negative classes
dv_sql_targets = "\n,".join(["sum({dv}) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_targets = f"""select {dv_sql_targets} from {DV_TABLE};"""
sql_count_targets = civis.io.read_civis_sql(sql_collapse_targets, DATABASE)

dv_sql_zeroes = "\n,".join(["sum(case when {dv} = 0 then 1 end) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_zeroes = f"""select {dv_sql_zeroes} from {DV_TABLE};"""
sql_count_zeroes = civis.io.read_civis_sql(sql_collapse_zeroes, DATABASE)


In [6]:
# Determing training table proportion of positives to negatives (to avoid class imbalance problems)
sample_share = []
for i in range(len(DV_LIST)):
    u = round((int(sql_count_targets[1][i])*2)/(int(sql_count_zeroes[1][i])),5)
    sample_share.append(u)

In [7]:
# Create training views
for i in range(len(DV_LIST)):
    if (int(sql_count_targets[1][i])*3) <= 1000:
        feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_small])
    if (int(sql_count_targets[1][i])*3) > 1000 & (int(sql_count_targets[1][i])*3) <= 2000:
        feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_medium])
    if (int(sql_count_targets[1][i])*3) > 2000:
        feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_large])
    dv_item = DV_LIST[i]
    random_sample = sample_share[i]
    training_sql = f"""DROP VIEW IF EXISTS {SCHEMA}.{PREFIX}_training_{i} CASCADE;
    CREATE VIEW {SCHEMA}.{PREFIX}_training_{i} AS 
    (select * from (
    (select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 1) 
    union all 
    (select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 and random() < {random_sample}))
    left join
    (select {PRIMARY_KEY}, {feature_select} from {MODELING_FRAME}) using({PRIMARY_KEY}));"""
    create_training_sql = civis.io.query_civis(training_sql, database=DATABASE)
    create_training_sql.result().state
 

In [8]:
# Train models
train_list = []
model_list = []

for index, dv in enumerate(DV_LIST):
    print('TRAINING >>> {}'.format(dv))
    
    exc_list = DV_LIST.copy()
    exc_list.remove(dv)
    
    assert dv not in exc_list 
    
    name = f"""{dv}_{datestamp}"""
    model = ModelPipeline(model='sparse_logistic',
                          dependent_variable=dv,
                          primary_key=PRIMARY_KEY,
                          excluded_columns=EXCLUSION_COLUMNS,
                          calibration='sigmoid',
                          model_name=name,
                          memory_requested=12000)
    
    where_string = '{} is not null'.format(dv)

    train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training_{index}""", 
                        database_name=DATABASE,
                        sql_where=where_string#,
                        #fit_params={'sample_weight': WEIGHT_VAR}
                       )
    
    model_list.append(model)
    train_list.append(train)    


TRAINING >>> spoke_support_1box
TRAINING >>> spoke_persuasion_1plus
TRAINING >>> spoke_persuasion_1minus


In [9]:
# Extract successful models
model_output = model_list
train_output = train_list

jobs_list = []
for t in train_output: 
    try:
        if len(t.metadata['output']) > 0:  
            jobs_list.append(t)
            print('Job success')
    except:
        print('Job failure')
        pass

    
model_output, train_output = zip(*((m, t) for m, t in zip(model_output, train_output) if t in jobs_list))
model_output = list(model_output)
train_output = list(train_output)

Job success
Job success
Job success


In [10]:
# Generate validation metrics
metrics_list = []

for a, b in enumerate(train_output):
    metric = {'job_id':b.job_id,
              'run_id':b.run_id,
              'dv': ''.join(b.metadata['run']['configuration']['data']['y']),
              'model': b.metadata['model']['model'],
              'time_of_train_run': b.metadata['run']['time_of_run'],
              'n_rows': b.metadata['data']['n_rows'],
              'n_features': b.metadata['data']['n_cols'],
              'auc': b.metadata['metrics']['roc_auc'],
              'deciles': b.metadata['metrics']['deciles'],
              'confusion_matrix': b.metadata['metrics']['confusion_matrix'],
              'accuracy': b.metadata['metrics']['accuracy'],
              'p_correct': b.metadata['metrics']['p_correct'],
              'pop_incidence_true': b.metadata['metrics']['pop_incidence_true'],
              'feature_list':b.metadata['model']['parameters']['relvars']
             }
    metrics_list.append(metric)
    
metric_order = (['job_id', 'run_id', 'dv', 'model', 'time_of_train_run', 'n_rows', 'n_features',
                 'auc', 'deciles', 'confusion_matrix', 'accuracy', 'p_correct','pop_incidence_true','feature_list'])

validation_df = pd.DataFrame.from_records(metrics_list, columns=metric_order, index='run_id')
validation_df

,job_id,dv,model,time_of_train_run,n_rows,n_features,auc,deciles,confusion_matrix,accuracy,p_correct,pop_incidence_true,feature_list
run_id,,,,,,,,,,,,,
192164893,53246690,spoke_support_1box,sparse_logistic,2019-12-18T15:51:32Z,162815,466,0.757561,"[0.056504114973590466, 0.10349487132240034, 0....","[[93206, 15300], [30502, 23807]]",0.718687,"[0.8589939726835383, 0.43836196578835923]","[0.6664373675644136, 0.3335626324355864]","[civis_2020_partisanship, civis_2018_gotv, civ..."
192164898,53246693,spoke_persuasion_1plus,sparse_logistic,2019-12-18T15:46:27Z,9140,466,0.624680,"[0.16849015317286653, 0.21663019693654267, 0.2...","[[5729, 319], [2803, 289]]",0.658425,"[0.947255291005291, 0.09346701164294954]","[0.661706783369803, 0.33829321663019696]","[civis_2020_ideology_liberal, civis_2018_gotv,..."
192164906,53246698,spoke_persuasion_1minus,sparse_logistic,2019-12-18T15:46:07Z,2643,466,0.722716,"[0.11363636363636363, 0.09433962264150944, 0.1...","[[1631, 160], [603, 249]]",0.711313,"[0.9106644332774986, 0.29225352112676056]","[0.677639046538025, 0.322360953461975]","[civis_2020_partisanship, civis_2020_race_whit..."


In [11]:
# Write validation metrics to Redshift
create_validation_table = civis.io.dataframe_to_civis(df=validation_df,
                                                 database=DATABASE, 
                                                 table= f'{SCHEMA}.{PREFIX}_validation_{datestamp}', 
                                                 existing_table_rows='drop')


In [12]:
# Score the voterfile
scores_list = []
for m,t in zip(model_output, train_output):
    DV_NAME = ''.join(t.metadata['run']['configuration']['data']['y'])
    print(DV_NAME)
    SCORES_TABLE = f'{SCHEMA}.{PREFIX}_{DV_NAME}_{datestamp}'
    scores_list.append(SCORES_TABLE)
    scores = m.predict(primary_key=PRIMARY_KEY,
                       database_name=DATABASE, 
                       table_name=MODELING_FRAME,
                       if_exists='drop',
                       output_table=SCORES_TABLE,
                       disk_space=20)
scores.result()


spoke_support_1box
spoke_persuasion_1plus
spoke_persuasion_1minus


{'container_id': 53249078,
 'error': None,
 'finished_at': '2019-12-18T18:20:12.000Z',
 'id': 192169675,
 'is_cancel_requested': False,
 'started_at': '2019-12-18T15:57:17.000Z',
 'state': 'succeeded'}

In [13]:
# Generate SQL for final output table and drop intermediary tables
view_list = []
table_list = []
for i in range(len(DV_LIST)):
    view = f"{SCHEMA}.{PREFIX}_training_{i}"
    view_list.append(view)
    table = f"{SCHEMA}.{PREFIX}_{DV_LIST[i]}_{datestamp}"
    table_list.append(table)

drop_view_sql = "\n".join(["drop view if exists {view} CASCADE;".format(view=v) for v in view_list])
drop_table_sql = "\n".join(["drop table if exists {tbl};".format(tbl=t) for t in table_list])  
dv_strings = "\n,".join(["{dv_score}_1 as {dv_score}".format(dv_score=dv) for dv in DV_LIST])
dv_tiles = "\n,".join(["NTILE(100) OVER (ORDER BY {dv_tile}_1) AS {dv_tile}_100".format(dv_tile=dv) for dv in DV_LIST])
join_table = []
if len(table_list) > 1:
    for i in table_list[1:]:
        j = str(' left join '+f'{i}'+f' using({PRIMARY_KEY}) ')
        join_table.append(j)
        #dv_strings = "\nleft join ".join(["{dv_score}".format(table=tbl) for tbl in table_list[i])


In [14]:
output_table_sql = f"""DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_output_{datestamp};
CREATE TABLE {SCHEMA}.{PREFIX}_output_{datestamp}
  DISTSTYLE KEY
  DISTKEY ({PRIMARY_KEY})
  SORTKEY ({PRIMARY_KEY})
  AS ("""+'select '+ f"{PRIMARY_KEY} \n," + dv_strings + "\n," + dv_tiles + ' from '+ ''.join(table_list[0]) + ''.join(join_table) +');'  


In [15]:
print(output_table_sql)

DROP TABLE IF EXISTS bernie_nmarchio2.scored_output_20191218;
CREATE TABLE bernie_nmarchio2.scored_output_20191218
  DISTSTYLE KEY
  DISTKEY (person_id)
  SORTKEY (person_id)
  AS (select person_id 
,spoke_support_1box_1 as spoke_support_1box
,spoke_persuasion_1plus_1 as spoke_persuasion_1plus
,spoke_persuasion_1minus_1 as spoke_persuasion_1minus
,NTILE(100) OVER (ORDER BY spoke_support_1box_1) AS spoke_support_1box_100
,NTILE(100) OVER (ORDER BY spoke_persuasion_1plus_1) AS spoke_persuasion_1plus_100
,NTILE(100) OVER (ORDER BY spoke_persuasion_1minus_1) AS spoke_persuasion_1minus_100 from bernie_nmarchio2.scored_spoke_support_1box_20191218 left join bernie_nmarchio2.scored_spoke_persuasion_1plus_20191218 using(person_id)  left join bernie_nmarchio2.scored_spoke_persuasion_1minus_20191218 using(person_id) );


In [16]:
# Create final output table
create_output_table = civis.io.query_civis(sql=output_table_sql, database=DATABASE)
create_output_table.result().state


'succeeded'

In [17]:
# Drop intermediary tables
drop_views_query = civis.io.query_civis(sql=drop_view_sql, database=DATABASE)
drop_views_query.result().state

drop_tables_query = civis.io.query_civis(sql=drop_table_sql, database=DATABASE)
drop_tables_query.result().state

'succeeded'

In [18]:
print(drop_view_sql)
print(drop_table_sql)

drop view if exists bernie_nmarchio2.scored_training_0 CASCADE;
drop view if exists bernie_nmarchio2.scored_training_1 CASCADE;
drop view if exists bernie_nmarchio2.scored_training_2 CASCADE;
drop table if exists bernie_nmarchio2.scored_spoke_support_1box_20191218;
drop table if exists bernie_nmarchio2.scored_spoke_persuasion_1plus_20191218;
drop table if exists bernie_nmarchio2.scored_spoke_persuasion_1minus_20191218;


In [19]:
# Grant team on tables
grant_statement = f"""
GRANT ALL ON SCHEMA {SCHEMA} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_output_{datestamp} TO GROUP bernie_data;
"""
grant_team = civis.io.query_civis(sql=grant_statement, database=DATABASE)
grant_team.result().state

'succeeded'

In [20]:
print(grant_statement)


GRANT ALL ON SCHEMA bernie_nmarchio2 TO GROUP bernie_data;
GRANT SELECT ON bernie_nmarchio2.scored_output_20191218 TO GROUP bernie_data;

